## Compilation



In [1]:
cd $HOME/models/lif_cpp
make clean
make

## Notebook



### Settings



    %load_ext autoreload
    %autoreload 2
    %reload_ext autoreload
    
    %run /home/leon/models/lif_cpp/notebooks/setup.py
    %matplotlib inline
    %config InlineBackend.figure_format = 'png'

    The autoreload extension is already loaded. To reload it, use:
      %reload_ext autoreload
    Python exe
    /home/leon/mambaforge/envs/dual_data/bin/python



### Imports



    import sys
    sys.path.insert(0, '/home/leon/models/lif_cpp')  
    REPO_PATH = "/home/leon/models/lif_cpp"
    import warnings
    warnings.simplefilter(action='ignore', category=FutureWarning)
    
    import subprocess
    import pandas as pd
    from time import sleep
    from yaml import safe_load
    from scipy.stats import circmean
    from joblib import Parallel, delayed
    
    from run_model import run_cpp, update_conf, restore_conf
    from analysis.decode import decode_bump, circcvl  
    pal = [sns.color_palette("tab10")[0], sns.color_palette("tab10")[1]]



### Helpers



#### Simulation



    def run_ini_phi(repo_path, conf_name, name, ini_list, phi_list, DURATION=10000, T_STEADY=5000, gain=1):
    
      conf_path = repo_path + '/conf/' + conf_name
      config = safe_load(open(conf_path + ".yml", "r"))
      data_path = repo_path + '/data/simul/'
    
      restore_conf(conf_path + '.yml',
                   conf_path +'.yml.bak')
    
      update_conf(conf_path, 'Iext', float(gain), axis=0)
      update_conf(conf_path, 'T_SAVE', float(0.0), axis=None)
      update_conf(conf_path, 'DURATION', float(DURATION), axis=None)
      update_conf(conf_path, 'T_STEADY', float(T_STEADY), axis=None)
    
      for ini in ini_list:
        for phi in phi_list:
          session = name + "_ini_%d_phi_%d" % (ini, phi)
          update_conf(conf_path, 'DATA_PATH', data_path + session)
          update_conf(conf_path, 'PHI_STIM', float(phi), axis=0)
    
          sleep(.2)
          run_cpp(session, conf_path = conf_path + ".yml")
          sleep(.2)
    
    
      restore_conf(conf_path + '.yml.bak',
                   conf_path +'.yml')



#### data



    def get_data(session, repo_path='/home/leon/models/lif_cpp'):
      # Open the file in binary mode and read the data
      with open(repo_path + '/data/simul/'+ session + '/rates.txt', 'rb') as f:
        data = np.fromfile(f, dtype=np.float32)
      # Now 'data' is a numpy array containing your data
      return data.reshape(-1, 40000).T[:32000]

    def get_rate(name, ini, phi):
         session = "/%s_ini_%d_phi_%d/" % (name, ini, phi)
         rates = get_data(session)
         return rates
    
    def get_rates_ini_phi(name, ini_list, phi_list):
         rates = Parallel(n_jobs=-1)(delayed(get_rate)(name, ini, phi) for ini in ini_list for phi in phi_list)
         rates = np.array(rates)
         rates = rates.reshape(len(ini_list), len(phi_list), rates.shape[1], -1)
         return rates

    def get_df_ini_phi(rates):
        n_trials, n_phi, n_neurons, n_times = rates.shape
    
        # Create indices
        trials_ind, phi_ind, neurons_ind, times_ind = np.indices((n_trials, n_phi, n_neurons, n_times))
    
        # Construct DataFrame
        df = pd.DataFrame({
            'trial': trials_ind.flatten(),
            'phi': phi_ind.flatten(),
            'neuron': neurons_ind.flatten(),
            'time': times_ind.flatten() * .250,
            'rates': rates.flatten()
        })
    
        return df

    def load_data_ini_phi(name, ini_list, phi_list):
        rates = get_rates_ini_phi(name, ini_list, phi_list)
        df = get_df_ini_phi(rates)
        return df

    def get_code(df):
        df_code = df.groupby(['time', 'trial', 'phi'])['rates'].apply(decode_bump).reset_index()
        df_code[['m0', 'm1', 'phase']] = pd.DataFrame(df_code['rates'].tolist(), index=df_code.index)
        df_code = df_code.drop(columns=['rates']).reset_index()
    
        end_point = df_code[df_code.time==df_code.time.iloc[-1]]
        end_point = end_point.drop(columns=['time'])
        return df_code, end_point

    def get_precision(x):
    
        cmean =  (x - circmean(x, low=-np.pi, high=np.pi)) 
    
        cmean[cmean > np.pi] -= 2*np.pi
        cmean[cmean < -np.pi] += 2*np.pi
    
        return cmean

    def get_times(conf_name):
         config = safe_load(open("/home/leon/models/lif_cpp/conf/"+conf_name+".yml", "r"))
    
         times = np.arange(0, 20000, config['T_WINDOW'])
         # times -= config['T_STEADY']
         times /= 1000.0
    
         return times



## Simulation



    conf_name = 'config'
    
    ini_list = np.arange(0, 1)
    print(ini_list)
    
    phi_list = np.linspace(0, 360, 9)[1:]
    print(phi_list)
    phi_list = [180]

    [0]
    [ 45.  90. 135. 180. 225. 270. 315. 360.]

    name = 'odr_off'
    run_ini_phi(REPO_PATH, conf_name, name, ini_list, phi_list, DURATION=5000, T_STEADY=1000, gain=2.0)



## Analysis



### data



    df = load_data_ini_phi(name, ini_list, phi_list)
    # print(df.head())
    df_code, end_point = get_code(df)
    df_code['phase'] *= 180 / np.pi
    
    end_point['precision'] = end_point.groupby(['phi'], group_keys=False)['phase'].apply(get_precision) * 180 / np.pi
    # end_point['phase'] *= 180 / np.pi

    df_smooth = df.groupby(['time', 'trial', 'phi'])['rates'].apply(circcvl).reset_index()
    
    smooth=[]
    point = df_smooth[df_smooth.time==df_smooth.time.iloc[-1]].reset_index()  
    for i in range(point.rates.shape[0]):
        m0, m1, phase = decode_bump(point.rates[i])
        smooth.append(np.roll(point.rates[i], int(( phase / 2.0 / np.pi - 0.5) * point.rates[i].shape[0])))
    
    smooth = np.array(smooth)

    fig, ax = plt.subplots(1, 1, figsize=(width, height))
    
    df_single = df[(df.phi==0) & (df.trial==0)]
    heatmap_df = df_single[['neuron', 'time', 'rates']]
    
    # Pivot the dataframe to get 'neurons' as rows, 'time' as columns, and 'rate' as values
    heatmap_data = heatmap_df.pivot(index='neuron', columns='time', values='rates')
    
    # Now, use seaborn to plot the heatmap
    sns.heatmap(heatmap_data, cmap='binary', vmin=0, vmax=10, cbar=None)
    ax.set_xticks([])  # Set the positions of the x-ticks
    ax.set_yticks([])  # Set the positions of the x-ticks
    
    # Show the plot
    plt.show()

![img](./.ob-jupyter/41e085b2e91a901228e4682c3d50f18cdc88da90.png)

    fig, ax = plt.subplots(2, 3, figsize=(3*width, 2*height))
    
    sns.lineplot(df_code, x='time', y='m0', legend=False, ax=ax[0][0])
    ax[0][0].set_xlabel('Time (s)')
    ax[0][0].set_ylabel('Firing Rate (Hz)')
    ax[0][0].set_ylim([0, 15])
    ax[0][0].set_xticks([0, 1, 2, 3, 4, 5])
    
    sns.lineplot(df_code, x='time', y=df_code['m1']/df_code['m0'], legend=False, ax=ax[0][1])
    ax[0][1].set_xlabel('Time (s)')
    ax[0][1].set_ylabel('Bump Amplitude, $\mathcal{F}_1$ (Hz)')
    ax[0][1].set_xticks([0, 1, 2, 3, 4, 5])
    
    f = lambda x: x
    
    categories = df_code[['trial', 'phi']].drop_duplicates()
    colors = plt.cm.jet(np.linspace(0, 1, len(categories)))
    for (trial, phi) in categories.values:
      # Select the subset of the data corresponding to the current trial and phi
      subset = df_code[(df_code['trial'] == trial) & (df_code['phi'] == phi)]
    
      # Plot the phase vs time for the current trial and phi
      ax[0][2].plot(subset['time'], subset['phase'], alpha=.25)
    
    ax[0][2].set_xlabel('Time (s)')
    ax[0][2].set_ylabel('Bump Center (°)')
    ax[0][2].set_ylim([-180, 180])
    ax[0][2].set_yticks([-180, -90, 0, 90, 180], [0, 90, 180, 270, 360])
    ax[0][2].set_xticks([0, 1, 2, 3, 4, 5])
    means = smooth.mean(axis=0)
    ci = smooth.std(axis=0, ddof=1) / np.sqrt(smooth.shape[0])
    
    # Plot
    ax[1][0].plot(means)
    ax[1][0].fill_between(range(smooth.shape[-1]), means - ci, means + ci, alpha=0.25)
    ax[1][0].set_xlabel('Neuron')
    ax[1][0].set_ylabel('Firing Rate (Hz)')
    
    sns.histplot(end_point, x=end_point.phase, ax=ax[1][1], bins='auto')
    ax[1][1].set_xlabel('Bump Center (°)')
    ax[1][1].set_xlim([-180, 180])
    
    sns.histplot(end_point, x=end_point.precision, ax=ax[1][2], bins='auto')
    ax[1][2].set_xlabel('Deviation (°)')
    # ax[1][2].set_xlim([-50, 50])
    plt.savefig('lif_m0.svg', dpi=300)
    plt.show()

![img](./.ob-jupyter/810ee61d7e2520e3a150e9a766b443ebdedfa3cb.png)

